In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from ncon import ncon
from scipy.linalg import polar
from scipy.sparse.linalg import eigs
from scipy.sparse.linalg import eigsh

In [2]:
def W_Ising(J,g):
    #Pauli and identity matrices 
    sx = 0.5*np.array([[0,1],[1,0]])  #Pauli x
    sz = 0.5*np.array([[1,0],[0,-1]]) #Pauli z
    I = np.eye(2)                     #Identity
    
    #Constructing the MPO at one site
    W = np.zeros((3,2,3,2))
    W[0,:,0,:] = I
    W[1,:,0,:] = sz
    W[2,:,0,:] = -g*sx
    W[2,:,1,:] = -J*sz
    W[2,:,2,:] = I
    return W

In [3]:
def W_XY():
    #Spin-1/2 operators and the identity
    su = np.array([[0,1],[0,0]])  #spin up
    sd = np.array([[0,0],[1,0]])  #spin down
    I = np.eye(2)                 #Identity

    #Constructing the MPO at one site
    W = np.zeros((4,2,4,2))
    W[0,:,0,:] = I
    W[1,:,0,:] = sd
    W[2,:,0,:] = su
    W[3,:,1,:] = -0.5*su
    W[3,:,2,:] = -0.5*sd
    W[3,:,3,:] = I
    
    return W

In [5]:
def Nsite_MPO(W,Nsites):
    """""
    Inputs :
        W : MPO at one site
        Nsites : number of desired sites
    Outputs :
        MPO : The N-site MPO
    """""
    #Produce the N-site MPO
    N = Nsites - 1
    MPO = [W for l in range(Nsites)]
    
    #Construct the states on the left and right side
    D_MPO = W.shape[0] #Bond dimension of the MPO
    state_0 = np.zeros((D_MPO,1))
    state_0[0,0] = 1
    state_1 = np.zeros((D_MPO,1))
    state_1[D_MPO-1,0] = 1
    
    #Modify the left- and right-most MPO site by contracting them with our states
    MPO[0] = ncon([MPO[0],state_0.T],[[1,-1,-2,-3],[-4,1]])
    MPO[0] = MPO[0].reshape(MPO[0].shape[0],MPO[0].shape[1],MPO[0].shape[2])
    MPO[N] = ncon([MPO[N],state_1],[[-1,-2,1,-3],[1,-4]])
    MPO[N] = MPO[N].reshape(MPO[N].shape[0],MPO[N].shape[1],MPO[N].shape[2])
    
    return MPO